# Decorateurs Python @
On peut décorer:
- classe
- fonctions (y compris méthode)

1 décorateur est une fonction qui enrichit l'objet décoré

In [110]:
from datetime import datetime
from functools import lru_cache, wraps
from statistics import  mean, stdev

## Intro avec lru_cache

In [2]:
def f(x):
    return x**2 + 1

In [3]:
f2 = lru_cache(f)
f2

In [4]:
f2(3)

10

In [7]:
@lru_cache
def g(x):
    return x**2 + 1

g

In [8]:
def fibo(n):
    match n:
        case 0:
            return 0
        case 1:
            return 1
        case _:
            return fibo(n-1) + fibo(n-2)

In [14]:
%timeit -n10 -r7 fibo(30)

437 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
@lru_cache
def fastfibo(n):
    match n:
        case 0:
            return 0
        case 1:
            return 1
        case _:
            return fastfibo(n-1) + fastfibo(n-2)

In [34]:
%timeit -n10 -r7 fastfibo(30)

The slowest run took 25.86 times longer than the fastest. This could mean that an intermediate result is being cached.
651 ns ± 1.21 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
fastfibo.cache_info()

CacheInfo(hits=97, misses=31, maxsize=128, currsize=31)

In [36]:
fastfibo(800)

69283081864224717136290077681328518273399124385204820718966040597691435587278383112277161967532530675374170857404743017623467220361778016172106855838975759985190398725

In [37]:
fastfibo.cache_info()

CacheInfo(hits=868, misses=801, maxsize=128, currsize=128)

In [38]:
@lru_cache(maxsize=500)
def ultrafastfibo(n):
    match n:
        case 0:
            return 0
        case 1:
            return 1
        case _:
            return ultrafastfibo(n-1) + ultrafastfibo(n-2)

In [40]:
fastfibo.cache_clear()
ultrafastfibo.cache_clear()

In [46]:
%timeit -n10 -r7 fastfibo(800)
%timeit -n10 -r7 ultrafastfibo(800)

The slowest run took 109.29 times longer than the fastest. This could mean that an intermediate result is being cached.
2.33 μs ± 5.3 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
The slowest run took 19.77 times longer than the fastest. This could mean that an intermediate result is being cached.
501 ns ± 845 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%timeit -n10 -r7 fastfibo(800)
%timeit -n10 -r7 ultrafastfibo(800)

233 ns ± 140 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)
221 ns ± 127 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
flashfibo = lru_cache(maxsize=800)(fibo)

In [48]:
# NB: seul le 1er appel est en cache, pas le recursif
%timeit -n10 -r7 flashfibo(30)
%timeit -n10 -r7 flashfibo(30)

The slowest run took 300923.84 times longer than the fastest. This could mean that an intermediate result is being cached.
6.02 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
149 ns ± 76.1 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
flashfibo.cache_parameters()

{'maxsize': 800, 'typed': False}

In [64]:
flashfibo?

Signature:       flashfibo(n)
Call signature:  flashfibo(*args, **kwargs)
Type:            _lru_cache_wrapper
String form:     <functools._lru_cache_wrapper object at 0x00000210F1244670>
File:            c:\users\matth\appdata\local\temp\ipykernel_15444\3324397206.py
Docstring:       <no docstring>
Class docstring:
Create a cached callable that wraps another function.

user_function:      the function being cached

maxsize:  0         for no caching
          None      for unlimited cache size
          n         for a bounded cache

typed:    False     cache f(3) and f(3.0) as identical calls
          True      cache f(3) and f(3.0) as distinct calls

cache_info_type:    namedtuple class with the fields:
                        hits misses currsize maxsize

## Decorateur custom
https://realpython.com/primer-on-python-decorators/

### Decorateur timing

In [77]:
def timing(f):
    @wraps(f) # from functools
    def wrapper(*args, **kwargs):
        dt1 = datetime.now()
        result = f(*args, **kwargs)
        dt2 = datetime.now()
        delta = dt2 - dt1
        print('Timing:', delta)
        return result
    # wrapper.__doc__ = f.__doc__
    # wrapper.__name__ = ...
    return wrapper

In [78]:
@timing
def compute():
    """Fait un super calcul qui prend du temps"""
    x = fibo(30)
    print('Fibo (30) =', x)

In [79]:
compute()

Fibo (30) = 832040
Timing: 0:00:00.465244


In [73]:
compute.__name__

'compute'

In [74]:
compute?

Signature: compute()
Docstring: Fait un super calcul qui prend du temps
File:      c:\users\matth\appdata\local\temp\ipykernel_15444\2133839334.py
Type:      function

In [80]:
@timing
def calcul(x):
    return 2**x + 3

In [81]:
calcul(3)

Timing: 0:00:00.000008


11

In [82]:
@timing
def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
    res = (a + b + c) * (d if e else -d) 
    if 'coef' in kwargs:
        res *= kwargs['coef']
    return res

In [86]:
super_computation(1, 2, 3, d=5, coef=0.9, dummy='dumber', e=False)

Timing: 0:00:00.000011


-27.0

In [87]:
values = [1, 2, 3]
params = {
    'd': 5, 'coef': 0.9, 'dummy': 'dumber', 'e': False
}

In [89]:
super_computation(*values, toto='tutu', **params)

Timing: 0:00:00.000010


-27.0

In [91]:
name_f = 'super_computation'
eval(name_f)(*values, toto='tutu', **params)

Timing: 0:00:00.000011


-27.0

### Décorateur avec paramètres

In [96]:
# comment ajouter un nb de fois à timing pour faire une moyenne et un ecart type
# @timing(10)

In [126]:
def timing(n: int = 1):
    
    def decorator(f):
    
        @wraps(f)
        def wrapper(*args, **kwargs):
            durations = []
            for _ in range(n):
                dt1 = datetime.now()
                result = f(*args, **kwargs)
                dt2 = datetime.now()
                delta = dt2 - dt1
                durations.append(delta.total_seconds()) 
            avg = mean(durations)
            std = stdev(durations) if n > 1 else 0.0
            print(f"Executed {n} times. Avg: {avg:.6f}s | Std Dev: {std:.6f}s")
            print('Timing:', delta)
            return result
        
        return wrapper
        
    if n <= 0:
        raise ValueError(f"n must be stritly positive, got {n}")
    return decorator

In [127]:
@timing(n=10)
def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
    res = (a + b + c) * (d if e else -d) 
    if 'coef' in kwargs:
        res *= kwargs['coef']
    return res

super_computation(1, 2, 3, d=5, coef=0.9, dummy='dumber', e=False)

Executed 10 times. Avg: 0.000004s | Std Dev: 0.000003s
Timing: 0:00:00.000002


-27.0

In [128]:
try:
    @timing(n=0)
    def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
        res = (a + b + c) * (d if e else -d) 
        if 'coef' in kwargs:
            res *= kwargs['coef']
        return res
except ValueError:
    print("ça n'est pas possible effectivement")

ça n'est pas possible effectivement


In [129]:
@timing()
def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
    res = (a + b + c) * (d if e else -d) 
    if 'coef' in kwargs:
        res *= kwargs['coef']
    return res

super_computation(1, 2, 3, d=5, coef=0.9, dummy='dumber', e=False)

Executed 1 times. Avg: 0.000010s | Std Dev: 0.000000s
Timing: 0:00:00.000010


-27.0

### Bonus appel decorator avec ou sans parenthèses

In [130]:
from dataclasses import dataclass
dataclass?

Signature:
dataclass(
    cls=None,
    /,
    *,
    init=True,
    repr=True,
    eq=True,
    order=False,
    unsafe_hash=False,
    frozen=False,
    match_args=True,
    kw_only=False,
    slots=False,
    weakref_slot=False,
)
Docstring:
Add dunder methods based on the fields defined in the class.

Examines PEP 526 __annotations__ to determine fields.

If init is true, an __init__() method is added to the class. If repr
is true, a __repr__() method is added. If order is true, rich
comparison dunder methods are added. If unsafe_hash is true, a
__hash__() method is added. If frozen is true, fields may not be
assigned to after instance creation. If match_args is true, the
__match_args__ tuple is added. If kw_only is true, then by default
all fields are keyword-only. If slots is true, a new class with a
__slots__ attribute is returned.
File:      c:\users\matth\miniconda3\envs\stage202510\lib\dataclasses.py
Type:      function

In [132]:
def timing(f=None, /, *, n: int = 1):
    
    def decorator(f):
    
        @wraps(f)
        def wrapper(*args, **kwargs):
            durations = []
            for _ in range(n):
                dt1 = datetime.now()
                result = f(*args, **kwargs)
                dt2 = datetime.now()
                delta = dt2 - dt1
                durations.append(delta.total_seconds()) 
            avg = mean(durations)
            std = stdev(durations) if n > 1 else 0.0
            print(f"Executed {n} times. Avg: {avg:.6f}s | Std Dev: {std:.6f}s")
            print('Timing:', delta)
            return result
        
        return wrapper
        
    if n <= 0:
        raise ValueError(f"n must be stritly positive, got {n}")
    if f is not None:
        return decorator(f)
    else:
        return decorator

In [133]:
@timing(n=10)
def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
    res = (a + b + c) * (d if e else -d) 
    if 'coef' in kwargs:
        res *= kwargs['coef']
    return res

super_computation(1, 2, 3, d=5, coef=0.9, dummy='dumber', e=False)

Executed 10 times. Avg: 0.000004s | Std Dev: 0.000003s
Timing: 0:00:00.000002


-27.0

In [134]:
try:
    @timing(n=0)
    def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
        res = (a + b + c) * (d if e else -d) 
        if 'coef' in kwargs:
            res *= kwargs['coef']
        return res
except ValueError:
    print("ça n'est pas possible effectivement")

ça n'est pas possible effectivement


In [135]:
@timing()
def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
    res = (a + b + c) * (d if e else -d) 
    if 'coef' in kwargs:
        res *= kwargs['coef']
    return res

super_computation(1, 2, 3, d=5, coef=0.9, dummy='dumber', e=False)

Executed 1 times. Avg: 0.000012s | Std Dev: 0.000000s
Timing: 0:00:00.000012


-27.0

In [136]:
@timing
def super_computation(a, b, c, /, *, d=3, e=True, **kwargs):
    res = (a + b + c) * (d if e else -d) 
    if 'coef' in kwargs:
        res *= kwargs['coef']
    return res

super_computation(1, 2, 3, d=5, coef=0.9, dummy='dumber', e=False)

Executed 1 times. Avg: 0.000011s | Std Dev: 0.000000s
Timing: 0:00:00.000011


-27.0